In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nept

from loading_data import get_data
from utils_maze import get_bin_centers, get_subset_zones, get_xy_idx

In [ ]:
thisdir = os.getcwd()
pickle_filepath = os.path.join(thisdir, "cache", "pickled")
output_filepath = os.path.join(thisdir, "plots", "trials")
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

In [ ]:
import info.r063d7 as info

In [ ]:
def trials_by_trajectory(info, sliced_position, zone, min_epoch=2., min_distance=10.):
    x_idxs, y_idxs = get_xy_idx(info, sliced_position)

    in_zone = zone[y_idxs, x_idxs]
    jumps = np.diff(in_zone.astype(int))
    jumps = np.insert(jumps,0,0)

    starts = sliced_position.time[jumps == 1]
    np.insert(starts, 0, sliced_position.time[0])
    stops = sliced_position.time[jumps == -1]
    np.insert(stops, 0, sliced_position.time[-1])

    zone_epochs = nept.Epoch([starts, stops])

    dur_idx = zone_epochs.durations >= min_epoch
    
    start_idxs = [nept.find_nearest_idx(sliced_position.time, start) for start in zone_epochs.starts]
    stop_idxs = [nept.find_nearest_idx(sliced_position.time, stop) for stop in zone_epochs.stops]

    dist_idx = np.zeros(zone_epochs.n_epochs).astype(bool)
    for i, (start_idx, stop_idx) in enumerate(zip(start_idxs, stop_idxs)):
        dist_idx[i] = sliced_position[start_idx].distance(sliced_position[stop_idx])[0] > min_distance

    return nept.Epoch([zone_epochs.starts[dur_idx&dist_idx], zone_epochs.stops[dur_idx&dist_idx]])

In [ ]:
events, position, spikes, lfp, _ = get_data(info)

u_zone, shortcut_zone, novel_zone = get_subset_zones(info, position)

sliced_position = position.time_slice(info.task_times["phase3"].start, info.task_times["phase3"].stop)

In [ ]:
zone_epochs = trials_by_trajectory(info, sliced_position, zone)

In [ ]:
for zone in [u_zone, shortcut_zone]:
    zone_epochs = trials_by_trajectory(info, sliced_position, zone)
    print(zone_epochs.n_epochs)
    tt = sliced_position.time_slice(zone_epochs.starts, zone_epochs.stops)

    plt.plot(position.x, position.y, "c.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()
    
    for trial_start, trial_stop in zip(zone_epochs.starts, zone_epochs.stops):

        tt = sliced_position.time_slice(trial_start, trial_stop)

        plt.plot(position.x, position.y, "y.")
        plt.plot(tt.x, tt.y, "k.", ms=4)
        plt.show()
        

zone_epochs = trials_by_trajectory(info, sliced_position, novel_zone, min_distance=0.)
print(zone_epochs.n_epochs)
tt = sliced_position.time_slice(zone_epochs.starts, zone_epochs.stops)

plt.plot(position.x, position.y, "c.")
plt.plot(tt.x, tt.y, "k.", ms=4)
plt.show()

for trial_start, trial_stop in zip(zone_epochs.starts, zone_epochs.stops):

    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
from utils_maze import get_trials

In [ ]:
phase = info.task_times["phase3"]
trials = get_trials(events, phase)

for trial_start, trial_stop in zip(trials.starts, trials.stops):

    tt = sliced_position.time_slice(trial_start, trial_stop)

    plt.plot(position.x, position.y, "y.")
    plt.plot(tt.x, tt.y, "k.", ms=4)
    plt.show()

In [ ]:
trials.n_epochs